In [1]:
import json
import os

import dotenv
import langchain
import langchain.chains
import langchain.chat_models
import langchain.llms
import langchain.memory
import langchain.prompts


# Load environment variables from .env file
dotenv.load_dotenv()

# Get the OpenAI API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is set
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in the .env file")



In [2]:
statements = [
    "∀x(Apple(x) → Fruit(x))",
    "∀x(Apple(x) ∧ Fruit(x))",
    "∃x(Apple(x) → Fruit(x))",
    "∃x(Apple(x) ∧ Fruit(x))",
    "∀x(Apple(x) ↔ Fruit(x))",
    "∀x(Fruit(x) → Apple(x))",
    "∃x(Fruit(x) ∧ Apple(x))",
    "∃x(Fruit(x) → Apple(x))",
    "∀x(Fruit(x) ↔ Apple(x))",
    "∃x(Fruit(x) ↔ Apple(x))",
    "∀x(Bird(x) → Fly(x))",
    "∀x(Bird(x) ∧ Fly(x))",
    "∃x(Bird(x) → Fly(x))",
    "∃x(Bird(x) ∧ Fly(x))",
    "∀x(Bird(x) ↔ Fly(x))",
    "∀x(Fly(x) → Bird(x))",
    "∃x(Fly(x) ∧ Bird(x))",
    "∃x(Fly(x) → Bird(x))",
    "∀x(Fly(x) ↔ Bird(x))",
    "∃x(Fly(x) ↔ Bird(x))",
    "∀x(Cat(x) → ¬Fish(x))",
    "∀x(Cat(x) ∧ ¬Fish(x))",
    "∃x(Cat(x) → ¬Fish(x))",
    "∃x(Cat(x) ∧ ¬Fish(x))",
    "∀x(Cat(x) ↔ ¬Fish(x))",
    "∀x(Fish(x) → ¬Cat(x))",
    "∃x(Fish(x) ∧ ¬Cat(x))",
    "∃x(Fish(x) → ¬Cat(x))",
    "∀x(Fish(x) ↔ ¬Cat(x))",
    "∃x(Fish(x) ↔ ¬Cat(x))",
    "∀x(Lion(x) → KingOfJungle(x))",
    "∃x(Lion(x) ∧ KingOfJungle(x))",
    "∀x(KingOfJungle(x) → Lion(x))",
    "∀x(Lion(x) ↔ KingOfJungle(x))",
    "∃x(Lion(x) → KingOfJungle(x))",
    "∀x(KingOfJungle(x) ∧ Lion(x))",
    "∃x(KingOfJungle(x) ↔ Lion(x))",
    "∃x(KingOfJungle(x) → Lion(x))",
    "∀x(KingOfJungle(x) ↔ ¬Lion(x))",
    "∃x(KingOfJungle(x) ∧ ¬Lion(x))",
    "∀x(Student(x) → Passed(x))",
    "∀x(Student(x) ∧ Passed(x))",
    "∃x(Student(x) → Passed(x))",
    "∃x(Student(x) ∧ Passed(x))",
    "∀x(Student(x) ↔ Passed(x))",
    "∀x(Passed(x) → Student(x))",
    "∃x(Passed(x) ∧ Student(x))",
    "∃x(Passed(x) → Student(x))",
    "∀x(Passed(x) ↔ ¬Student(x))",
    "∃x(Passed(x) ↔ Student(x))",
    "∀x(Dog(x) → Mammal(x)) ∧ ∃y(Mammal(y) ∧ ¬Dog(y))",
    "∀x(Dog(x) → Mammal(x)) ∧ ∀y(Mammal(y) → Dog(y))",
    "∀x(Dog(x) ∧ Mammal(x))",
    "∃x(Dog(x) → Mammal(x)) ∧ ∃y(Mammal(y) ∧ ¬Dog(y))",
    "∀x(Dog(x) ↔ Mammal(x))",
    "∀x(Mammal(x) → Dog(x))",
    "∃x(Mammal(x) ∧ ¬Dog(x))",
    "∃x(Mammal(x) → Dog(x))",
    "∀x(Mammal(x) ↔ ¬Dog(x))",
    "∃x(Dog(x) ∧ ¬Mammal(x))",
    "∀x(Lion(x) → KingOfJungle(x))",
    "∃x(Lion(x) ∧ KingOfJungle(x))",
    "∀x(KingOfJungle(x) → Lion(x))",
    "∀x(Lion(x) ↔ KingOfJungle(x))",
    "∃x(Lion(x) → KingOfJungle(x))",
    "∀x(KingOfJungle(x) ∧ Lion(x))",
    "∃x(KingOfJungle(x) ↔ Lion(x))",
    "∃x(KingOfJungle(x) → Lion(x))",
    "∀x(KingOfJungle(x) ↔ ¬Lion(x))",
    "∃x(KingOfJungle(x) ∧ ¬Lion(x))",
    "∀x(Student(x) → Passed(x))",
    "∀x(Student(x) ∧ Passed(x))",
    "∃x(Student(x) → Passed(x))",
    "∃x(Student(x) ∧ Passed(x))",
    "∀x(Student(x) ↔ Passed(x))",
    "∀x(Passed(x) → Student(x))",
    "∃x(Passed(x) ∧ Student(x))",
    "∃x(Passed(x) → Student(x))",
    "∀x(Passed(x) ↔ ¬Student(x))",
    "∃x(Passed(x) ↔ Student(x))",
    "∀x(Dog(x) → Mammal(x)) ∧ ∃y(Mammal(y) ∧ ¬Dog(y))",
    "∀x(Dog(x) → Mammal(x)) ∧ ∀y(Mammal(y) → Dog(y))",
    "∀x(Dog(x) ∧ Mammal(x))",
    "∃x(Dog(x) → Mammal(x)) ∧ ∃y(Mammal(y) ∧ ¬Dog(y))",
    "∀x(Dog(x) ↔ Mammal(x))",
    "∀x(Mammal(x) → Dog(x))",
    "∃x(Mammal(x) ∧ ¬Dog(x))",
    "∃x(Mammal(x) → Dog(x))",
    "∀x(Mammal(x) ↔ ¬Dog(x))",
    "∃x(Dog(x) ∧ ¬Mammal(x))",
    "∀x(Student(x) → LoveMath(x))",
    "∀x(Student(x) ∧ LoveMath(x))",
    "∃x(Student(x) → LoveMath(x))",
    "∃x(Student(x) ∧ LoveMath(x))",
    "∀x(Student(x) ↔ LoveMath(x))",
    "∀x(LoveMath(x) → Student(x))",
    "∃x(LoveMath(x) ∧ Student(x))",
    "∃x(LoveMath(x) → Student(x))",
    "∀x(LoveMath(x) ↔ ¬Student(x))",
    "∃x(LoveMath(x) ↔ Student(x))",
    "∀x(Thunderstorm(x) → Rain(x))",
    "∀x(Thunderstorm(x) ∧ Rain(x))",
    "∃x(Thunderstorm(x) → Rain(x))",
    "∃x(Thunderstorm(x) ∧ Rain(x))",
    "∀x(Thunderstorm(x) ↔ Rain(x))",
    "∀x(Rain(x) → Thunderstorm(x))",
    "∃x(Rain(x) ∧ Thunderstorm(x))",
    "∃x(Rain(x) → Thunderstorm(x))",
    "∀x(Rain(x) ↔ ¬Thunderstorm(x))",
    "∃x(Rain(x) ↔ Thunderstorm(x))",
    "∀x(Integer(x) → (Even(x) ∨ Odd(x)))",
    "∀x(Integer(x) ∧ (Even(x) ∨ Odd(x)))",
    "∃x(Integer(x) → (Even(x) ∨ Odd(x)))",
    "∃x(Integer(x) ∧ (Even(x) ∨ Odd(x)))",
    "∀x(Integer(x) ↔ (Even(x) ∨ Odd(x)))",
    "∀x((Even(x) ∨ Odd(x)) → Integer(x))",
    "∃x((Even(x) ∨ Odd(x)) ∧ Integer(x))",
    "∃x((Even(x) ∨ Odd(x)) → Integer(x))",
    "∀x((Even(x) ∨ Odd(x)) ↔ ¬Integer(x))",
    "∃x((Even(x) ∨ Odd(x)) ↔ Integer(x))",
    "∀x(Song(x) → Instrumental(x))",
    "∀x(Song(x) ∧ Instrumental(x))",
    "∃x(Song(x) → Instrumental(x))",
    "∃x(Song(x) ∧ Instrumental(x))",
    "∀x(Song(x) ↔ Instrumental(x))",
    "∀x(Instrumental(x) → Song(x))",
    "∃x(Instrumental(x) ∧ Song(x))",
    "∃x(Instrumental(x) → Song(x))",
    "∀x(Instrumental(x) ↔ ¬Song(x))",
    "∃x(Instrumental(x) ↔ Song(x))",
    "∀x(Musician(x) → Guitar(x))",
    "∀x(Musician(x) ∧ Guitar(x))",
    "∃x(Musician(x) → Guitar(x))",
    "∃x(Musician(x) ∧ Guitar(x))",
    "∀x(Musician(x) ↔ Guitar(x))",
    "∀x(Guitar(x) → Musician(x))",
    "∃x(Guitar(x) ∧ Musician(x))",
    "∃x(Guitar(x) → Musician(x))",
    "∀x(Guitar(x) ↔ ¬Musician(x))",
    "∃x(Guitar(x) ↔ Musician(x))",
    "∀x(Rose(x) → Flower(x))",
    "∀x(Rose(x) ∧ Flower(x))",
    "∃x(Rose(x) → Flower(x))",
    "∃x(Rose(x) ∧ Flower(x))",
    "∀x(Rose(x) ↔ Flower(x))",
    "∀x(Flower(x) → Rose(x))",
    "∃x(Flower(x) ∧ Rose(x))",
    "∃x(Flower(x) → Rose(x))",
    "∀x(Flower(x) ↔ ¬Rose(x))",
    "∃x(Flower(x) ↔ Rose(x))",
    "∀x(Number(x) → (Prime(x) ∧ Odd(x)))",
    "∀x(Number(x) ∧ (Prime(x) ∧ Odd(x)))",
    "∃x(Number(x) → (Prime(x) ∧ Odd(x)))",
    "∃x(Number(x) ∧ (Prime(x) ∧ Odd(x)))",
    "∀x(Number(x) ↔ (Prime(x) ∧ Odd(x)))",
    "∀x((Prime(x) ∧ Odd(x)) → Number(x))",
    "∃x((Prime(x) ∧ Odd(x)) ∧ Number(x))",
    "∃x((Prime(x) ∧ Odd(x)) → Number(x))",
    "∀x((Prime(x) ∧ Odd(x)) ↔ ¬Number(x))",
    "∃x((Prime(x) ∧ Odd(x)) ↔ Number(x))",
    "∀x(Circle(x) → Round(x))",
    "∀x(Circle(x) ∧ Round(x))",
    "∃x(Circle(x) → Round(x))",
    "∃x(Circle(x) ∧ Round(x))",
    "∀x(Circle(x) ↔ Round(x))",
    "∀x(Round(x) → Circle(x))",
    "∃x(Round(x) ∧ Circle(x))",
    "∃x(Round(x) → Circle(x))",
    "∀x(Round(x) ↔ ¬Circle(x))",
    "∃x(Round(x) ↔ Circle(x))",
    "∀x(Lake(x) → Water(x))",
    "∀x(Lake(x) ∧ Water(x))",
    "∃x(Lake(x) → Water(x))",
    "∃x(Lake(x) ∧ Water(x))",
    "∀x(Lake(x) ↔ Water(x))",
    "∀x(Water(x) → Lake(x))",
    "∃x(Water(x) ∧ Lake(x))",
    "∃x(Water(x) → Lake(x))",
    "∀x(Water(x) ↔ ¬Lake(x))",
    "∃x(Water(x) ↔ Lake(x))",
    "∀x(Friendship(x) → Lifetime(x))",
    "∀x(Friendship(x) ∧ Lifetime(x))",
    "∃x(Friendship(x) → Lifetime(x))",
    "∃x(Friendship(x) ∧ Lifetime(x))",
    "∀x(Friendship(x) ↔ Lifetime(x))",
    "∀x(Lifetime(x) → Friendship(x))",
    "∃x(Lifetime(x) ∧ Friendship(x))",
    "∃x(Lifetime(x) → Friendship(x))",
    "∀x(Lifetime(x) ↔ ¬Friendship(x))",
    "∃x(Lifetime(x) ↔ Friendship(x))",
    "∀x(Set(x) → Member(x))",
    "∀x(Set(x) ∧ Member(x))",
    "∃x(Set(x) → Member(x))",
    "∃x(Set(x) ∧ Member(x))",
    "∀x(Set(x) ↔ Member(x))",
    "∀x(Member(x) → Set(x))",
    "∃x(Member(x) ∧ Set(x))",
    "∃x(Member(x) → Set(x))",
    "∀x(Member(x) ↔ ¬Set(x))",
    "∃x(Member(x) ↔ Set(x))",
    "∀x(WinterDay(x) → Cold(x))",
    "∀x(WinterDay(x) ∧ Cold(x))",
    "∃x(WinterDay(x) → Cold(x))",
    "∃x(WinterDay(x) ∧ Cold(x))",
    "∀x(WinterDay(x) ↔ Cold(x))",
    "∀x(Cold(x) → WinterDay(x))",
    "∃x(Cold(x) ∧ WinterDay(x))",
    "∃x(Cold(x) → WinterDay(x))",
    "∀x(Cold(x) ↔ ¬WinterDay(x))",
    "∃x(Cold(x) ↔ WinterDay(x))",
    "∀x(Student(x) ↔ Study(x))",
    "∀x(Study(x) → Student(x))",
    "∃x(Student(x) ∧ Study(x))",
    "∀x(Student(x) ∧ Study(x))",
    "∀x(Study(x) ↔ ¬Student(x))",
    "∃x(Study(x) → Student(x))",
    "∀x(¬Student(x) → ¬Study(x))",
    "∃x(Study(x) ↔ Student(x))",
    "∀x(Student(x) → Study(x))",
    "∃x(¬Student(x) ∧ Study(x))",
    "∃x(Bird(x) ∧ ¬Fly(x))",
    "∀x(Bird(x) → ¬Fly(x))",
    "∃x(Bird(x) → Fly(x))",
    "∀x(¬Bird(x) → Fly(x))",
    "∃x(Fly(x) ∧ ¬Bird(x))",
    "∀x(Bird(x) → Fly(x))",
    "∃x(¬Bird(x) → Fly(x))",
    "∀x(Fly(x) → Bird(x))",
    "∀x(¬Fly(x) → Bird(x))",
    "∀x(Fly(x) ↔ Bird(x))",
    "¬Play(Alice, Guitar) ∧ ¬Play(Bob, Guitar)",
    "¬Play(Alice, Guitar) → ¬Play(Bob, Guitar)",
    "Play(Alice, Guitar) ∧ Play(Bob, Guitar)",
    "Play(Alice, Guitar) → Play(Bob, Guitar)",
    "¬Play(Alice, Guitar) ↔ ¬Play(Bob, Guitar)",
    "Play(Alice, Guitar) ↔ ¬Play(Bob, Guitar)",
    "¬Play(Alice, Guitar) ∨ ¬Play(Bob, Guitar)",
    "Play(Alice, Guitar) ∨ Play(Bob, Guitar)",
    "¬Play(Alice, Guitar) → Play(Bob, Guitar)",
    "Play(Alice, Guitar) → ¬Play(Bob, Guitar)",
    "Rain(x) ↔ ¬Snow(x)",
    "Rain(x) ∧ Snow(x)",
    "Rain(x) → ¬Snow(x)",
    "¬Rain(x) → Snow(x)",
    "Rain(x) ∨ Snow(x)",
    "Rain(x) ↔ Snow(x)",
    "¬Rain(x) ∧ ¬Snow(x)",
    "Rain(x) → Snow(x)",
    "¬Rain(x) ↔ ¬Snow(x)",
    "Rain(x) ∧ ¬Snow(x)",
    "∀x(Musician(x) → ¬Hate(x, Music))",
    "∃x(Musician(x) ∧ Hate(x, Music))",
    "∀x(Musician(x) ↔ ¬Hate(x, Music))",
    "∃x(Musician(x) → ¬Hate(x, Music))",
    "∀x(¬Musician(x) → Hate(x, Music))",
    "∀x(Hate(x, Music) → ¬Musician(x))",
    "∃x(Hate(x, Music) ∧ Musician(x))",
    "∃x(¬Musician(x) → Hate(x, Music))",
    "∀x(Musician(x) ∧ ¬Hate(x, Music))",
    "∃x(Hate(x, Music) ↔ Musician(x))",
    "∀x(Love(x, Sun) → Go(x, Beach))",
    "∀x(Go(x, Beach) → Love(x, Sun))",
    "∃x(Love(x, Sun) ∧ Go(x, Beach))",
    "∀x(Go(x, Beach) ↔ Love(x, Sun))",
    "∀x(¬Go(x, Beach) ↔ ¬Love(x, Sun))",
    "∀x(Go(x, Beach) ∧ Love(x, Sun))",
    "∃x(Go(x, Beach) → Love(x, Sun))",
    "∀x(Love(x, Sun) ↔ Go(x, Beach))",
    "∃x(¬Love(x, Sun) ↔ ¬Go(x, Beach))",
    "∀x(Love(x, Sun) ∧ Go(x, Beach))",
    "∃x(Number(x) ∧ Odd(x) ∧ Prime(x))",
    "∀x(Number(x) → (Odd(x) ∧ Prime(x)))",
    "∃x(Prime(x) → Odd(x))",
    "∀x(Odd(x) ↔ Prime(x))",
    "∃x(Odd(x) ↔ Prime(x))",
    "∃x(Number(x) → (Odd(x) ∧ Prime(x)))",
    "∀x(Number(x) ∧ Odd(x) → Prime(x))",
    "∃x(Number(x) ∧ Prime(x) → Odd(x))",
    "∀x(Prime(x) → Odd(x))",
    "∃x(Number(x) ∧ Prime(x) ∧ ¬Odd(x))",
    "∃x(Animal(x) ∧ Fly(x) ∧ ¬Bird(x))",
    "∀x(Animal(x) ∧ Fly(x) → ¬Bird(x))",
    "∃x(Animal(x) ∧ Bird(x) → ¬Fly(x))",
    "∀x(Animal(x) ∧ ¬Fly(x) → Bird(x))",
    "∃x(Animal(x) ∧ Fly(x) ↔ Bird(x))",
    "∀x(Animal(x) ∧ Fly(x) ↔ ¬Bird(x))",
    "∀x(Animal(x) → (Fly(x) ↔ Bird(x)))",
    "∃x(Animal(x) → (Fly(x) ∧ ¬Bird(x)))",
    "∀x(Animal(x) → (Fly(x) → Bird(x)))",
    "∃x(Bird(x) ∧ Fly(x) ∧ Animal(x))",
    "∀x(Sing(x) → Bird(x))",
    "∀x(Bird(x) → Sing(x))",
    "∃x(Bird(x) ∧ Sing(x))",
    "∀x(Bird(x) ↔ Sing(x))",
    "∀x(Sing(x) ↔ ¬Bird(x))",
    "∃x(Sing(x) → Bird(x))",
    "∃x(Bird(x) ∧ ¬Sing(x))",
    "∃x(Bird(x) ↔ Sing(x))",
    "∀x(¬Bird(x) → ¬Sing(x))",
    "∃x(¬Bird(x) ∧ Sing(x))",
    "∃x(Prime(x) ∧ Even(x))",
    "∀x(Prime(x) → Even(x))",
    "∃x(Prime(x) ↔ Even(x))",
    "∀x(Prime(x) ↔ Even(x))",
    "∃x(Even(x) → Prime(x))",
    "∀x(Even(x) → Prime(x))",
    "∃x(Prime(x) ∧ ¬Even(x))",
    "∀x(Prime(x) ∧ Even(x))",
    "∃x(¬Prime(x) ↔ Even(x))",
    "∀x(¬Prime(x) ↔ Even(x))",
    "∀x(Poet(x) → Appreciate(x, Beauty) ∧ ¬Write(x, Beauty))",
    "∀x(Poet(x) → Appreciate(x, Beauty)) ∧ ∃x(Poet(x) ∧ ¬Write(x, Beauty))",
    "∃x(Poet(x) ∧ Appreciate(x, Beauty) ∧ Write(x, Beauty))",
    "∀x(Poet(x) ↔ (Appreciate(x, Beauty) ∧ ¬Write(x, Beauty)))",
    "∀x(Poet(x) → Write(x, Beauty)) ∧ ∃x(Poet(x) ∧ ¬Appreciate(x, Beauty))",
    "∃x(Poet(x) ↔ (Appreciate(x, Beauty) ∧ Write(x, Beauty)))",
    "∀x(Poet(x) → (Appreciate(x, Beauty) ↔ Write(x, Beauty)))",
    "∀x(Poet(x) → Appreciate(x, Beauty)) ∧ ∀x(Poet(x) → ¬Write(x, Beauty))",
    "∃x(Poet(x) → (Appreciate(x, Beauty) ∧ Write(x, Beauty)))",
    "∀x(Poet(x) ↔ Appreciate(x, Beauty))",
    "∃x(Musician(x) ∧ Play(x, Piano) ∧ Play(x, Violin))",
    "∀x(Musician(x) → (Play(x, Piano) ∧ Play(x, Violin)))",
    "∃x(Musician(x) ↔ (Play(x, Piano) ∧ Play(x, Violin)))",
    "∀x(Musician(x) ↔ (Play(x, Piano) ∧ Play(x, Violin)))",
    "∃x(Musician(x) → (Play(x, Piano) ∧ Play(x, Violin)))",
    "∃x(Musician(x) ∧ Play(x, Piano) → Play(x, Violin))",
    "∀x(Musician(x) ∧ Play(x, Piano) → Play(x, Violin))",
    "∃x(Musician(x) ∧ (Play(x, Piano) ↔ Play(x, Violin)))",
    "∀x(Musician(x) ∧ (Play(x, Piano) ↔ Play(x, Violin)))",
    "∀x(Musician(x) → Play(x, Piano) ↔ Play(x, Violin))",
    "∀x∀y(Even(x) ∧ Even(y) ↔ Even(x+y))",
    "∀x∀y(Even(x) ∧ Even(y) → Even(x+y))",
    "∃x∃y(Even(x) ∧ Even(y) ↔ Even(x+y))",
    "∀x∀y(Even(x+y) → Even(x) ∧ Even(y))",
    "∃x∃y(Even(x) ∧ Even(y) → Even(x+y))",
    "∀x∀y(Even(x+y) ↔ Even(x) ∧ Even(y))",
    "∀x∀y(Even(x) → Even(y) ↔ Even(x+y))",
    "∃x∃y(Even(x+y) ↔ Even(x) ∧ Even(y))",
    "∀x∀y(¬Even(x+y) ↔ ¬Even(x) ∧ ¬Even(y))",
    "∀x∀y(Even(x) ∧ ¬Even(y) ↔ ¬Even(x+y))",
    "∀x∀y(Predator(x,y) → ¬Predator(y,x))",
    "∀x∀y(Predator(x,y) ↔ ¬Predator(y,x))",
    "∃x∃y(Predator(x,y) → ¬Predator(y,x))",
    "∃x∃y(Predator(x,y) ↔ Predator(y,x))",
    "∀x∀y(¬Predator(x,y) → Predator(y,x))",
    "∀x∀y(Predator(x,y) ∧ ¬Predator(y,x))",
    "∀x∀y(¬Predator(x,y) ↔ ¬Predator(y,x))",
    "∃x∃y(Predator(x,y) ∧ ¬Predator(y,x))",
    "∀x∀y(Predator(x,y) ↔ Predator(y,x))",
    "∃x∃y(¬Predator(x,y) ↔ ¬Predator(y,x))",
    "∃x∃y(Helps(x,y) → ¬Helps(y,x))",
    "∃x∃y(Helps(x,y) ↔ ¬Helps(y,x))",
    "∀x∀y(Helps(x,y) → ¬Helps(y,x))",
    "∀x∀y(Helps(x,y) ↔ ¬Helps(y,x))",
    "∃x∃y(Helps(x,y) ∧ Helps(y,x))",
    "∃x∃y(Helps(x,y) ∧ ¬Helps(y,x))",
    "∀x∀y(Helps(x,y) ∧ ¬Helps(y,x))",
    "∀x∀y(¬Helps(x,y) ↔ Helps(y,x))",
    "∀x∀y(Helps(x,y) → Helps(y,x))",
    "∀x∀y(¬Helps(x,y) → ¬Helps(y,x))",
    "∀x∀y(Congruent(x,y) → SameArea(x,y))",
    "∀x∀y(Congruent(x,y) ↔ SameArea(x,y))",
    "∃x∃y(Congruent(x,y) → SameArea(x,y))",
    "∀x∀y(SameArea(x,y) → Congruent(x,y))",
    "∃x∃y(Congruent(x,y) ↔ SameArea(x,y))",
    "∀x∀y(Congruent(x,y) ∧ SameArea(x,y))",
    "∀x∀y(¬Congruent(x,y) ↔ ¬SameArea(x,y))",
    "∃x∃y(Congruent(x,y) ∧ ¬SameArea(x,y))",
    "∀x∀y(Congruent(x,y) ↔ ¬SameArea(x,y))",
    "∃x∃y(¬Congruent(x,y) ↔ SameArea(x,y))",

]

In [20]:
#################################################################
## EXAMPLE 2: Generate MULTIPLE responses with the GPT-3 engine #
#################################################################

# Initialize the OpenAI object
#llm = langchain.llms.OpenAI(openai_api_key=openai_api_key, model="text-davinci-003")
llm = langchain.chat_models.ChatOpenAI(model_name="gpt-4", temperature=0.3)

# Define the prompt template

template = """You are going to be asked to expand logical prompts into English statements and negations. The output should be in JSON.

For instance, the first of the following prompts
```
    "∀x(Apple(x) → Fruit(x))",
    "∀x(Apple(x) ∧ Fruit(x))",
    "∃x(Apple(x) ∧ Fruit(x))",
```
correspond to a record with the following fields, respectively:
```json
[
        {{ 
            "statement": "∀x(Apple(x) → Fruit(x))", 
            "english": "For every object, if it's an apple, then it's a fruit.", 
            "englishTerse": "All apples are fruits.",
            "negateStatement": "∃x(Apple(x) ∧ ¬Fruit(x))",
            "negateEnglish": "There exists an object that is an apple but not a fruit.",
            "negateEnglishTerse": "Some apples are not fruits.",
            "predicates": ["Apple", "Fruit"],
            "topics": ["Universal", "Implication"]
            "confidence": 100.0
        }},
]
```
Make sure of the following:
- The "english" key must translate the logic expression contained in "statement" into English, as precisely as possible. Be careful about using "if ... then ..." in the wrong context. For instance, "∀x(Apple(x) → Fruit(x))" should be translated as "For every object, if it's an apple, then it's a fruit." and not "If it's an apple, then it's a fruit." For instance, "∀x(Apple(x) ∧ Fruit(x))" should be translated as "For every object, it is an apple and a fruit." or "Every object is an apple and a fruit."
- The "englishTerse" key must translate the logic expression contained in "statement" into English, as tersely as possible. For instance, "∀x(Apple(x) → Fruit(x))" should be translated as "All apples are fruits." and not "For every object, if it's an apple, then it's a fruit." For instance, "∀x(Apple(x) ∧ Fruit(x))" should be translated as "Everything is an apple and a fruit."
- Keep Negations as Short as Possible: For instance, the correct negation of "∀x(Apple(x) ∧ Fruit(x))" is "∃x(¬Apple(x) ∨ ¬Fruit(x))" (applying De Morgan's law), whereas "∃x(Apple(x) ∧ ¬Fruit(x)) ∨ ∃x(¬Apple(x) ∧ Fruit(x))" is correct but more verbose than necessary.

Now it's your turn. If you are unsure.

Statement: {statement}
Provide output as JSON: {{ 
    "statement": "{statement}", 
    "english": "<to be completed>", 
    "englishTerse": "<to be completed>",
    "negateStatement": "<to be completed>",
    "negateEnglish": "<to be completed>",
    "negateEnglishTerse": "<to be completed>",
    "predicates": "<to be completed>",
    "topics": "<to be completed>",
    "confidence": "<your confidence with the response, between 0 and 100.0"
}}
"""
prompt = langchain.prompts.PromptTemplate(
    template=template, input_variables=["statement"]
)


# Load cache
statements_processed = {}
if os.path.exists("statements.json"):
    try:
        statements_processed = json.load(open("statements.json"))
    except:
        pass


llm_chain = langchain.chains.LLMChain(
    prompt=prompt,
    llm=llm,
    verbose=True)

# Batch generation
# questions = [
#     {"statement": statement}
#     for statement in statements
# ]

# response = llm_chain.generate(questions)
import tqdm
for statement in tqdm.tqdm(statements):
    if statement in statements_processed and statements_processed.get(statement, "") != "":
        continue

    try:
        response = llm_chain.generate([{"statement": statement}])
    except:
        raise
        continue
    
    statement_expanded_json = "\n".join(
        blurb.text.strip()
        for generation in response.generations
        for blurb in generation
    )

    statement_expanded = "ERROR CONVERTING <{}>".format(statement_expanded_json)
    try:
        statement_expanded = json.loads(statement_expanded_json)
    except:
        pass
    statements_processed[statement] = statement_expanded

    # Save cache
    try:
        open("statements.json", "w").write(json.dumps(statements_processed, indent=2))
    except Exception as e:
        print(e)
        pass

# response_text = [
#     json.loads(blurb.text.strip())
#     for generation in response.generations
#     for blurb in generation
# ]


  0%|          | 0/370 [00:00<?, ?it/s]

  1%|          | 2/370 [00:44<2:17:56, 22.49s/it]


KeyboardInterrupt: 

In [12]:
import json
open("temp.json", "w").write(json.dumps(response_text, indent=2))

3133